[LangChain Cookbook Part2](https://github.com/ea-datascience/langchain-tutorials/blob/main/LangChain%20Cookbook%20Part%202%20-%20Use%20Cases.ipynb)

In [1]:
import os
from dotenv import load_dotenv

# Initialize the OpenAI API
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

open_ai_key = os.environ["OPENAI_API_KEY"]
google_cse_id = os.environ["GOOGLE_CSE_ID"]
google_api_key = os.environ["GOOGLE_API_KEY"]

## Use cases

1. Summarization
2. Question and Answering over documents
3. Extraction
4. Evaluation
5. Querying Tabular Data
6. Code understanding
7. Interacting with APIs
8. Chatbots
9. Agents

### 1. SUMMARIZATION

Summaries

In [2]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

# Note that what is the default model 'text-davinci-003'
llm = OpenAI(temperature = 0, model_name = 'text-davinci-003', openai_api_key = open_ai_key)

# Create our template
template = """
% INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

%TEXT:
{text}
"""

# Create a LangChain prompt template that we can insert values to later
prompt = PromptTemplate(
    input_variables = ["text"],
    template = template,
)

In [3]:
confusing_text = """
For the next 130 years, debate raged.
Some scientists called Prototaxites a lichen, others a fungus, and still others clung to the notion that it was some kind of tree.
“The problem is that when you look up close at the anatomy, it’s evocative of a lot of different things, but it’s diagnostic of nothing,” says Boyce, an associate professor in geophysical sciences and the Committee on Evolutionary Biology.
“And it’s so damn big that when whenever someone says it’s something, everyone else’s hackles get up: ‘How could you have a lichen 20 feet tall?’”
"""

In [4]:
print("------- Prompt Begin -------")

final_prompt = prompt.format(text = confusing_text)
print(final_prompt)

print("------- Prompt End -------")

------- Prompt Begin -------

% INSTRUCTIONS:
Please summarize the following piece of text.
Respond in a manner that a 5 year old would understand.

%TEXT:

For the next 130 years, debate raged.
Some scientists called Prototaxites a lichen, others a fungus, and still others clung to the notion that it was some kind of tree.
“The problem is that when you look up close at the anatomy, it’s evocative of a lot of different things, but it’s diagnostic of nothing,” says Boyce, an associate professor in geophysical sciences and the Committee on Evolutionary Biology.
“And it’s so damn big that when whenever someone says it’s something, everyone else’s hackles get up: ‘How could you have a lichen 20 feet tall?’”


------- Prompt End -------


In [5]:
output = llm(final_prompt)
print(output)


For 130 years, people argued about what Prototaxites was. Some thought it was a lichen, some thought it was a fungus, and some thought it was a tree. But no one could agree because when you looked closely, it looked like a lot of different things. It was also very big, so people were surprised that it could be a lichen.


## 1.2 SUMMARIES OF LONGER TEXT

In [6]:
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

llm = OpenAI(temperature = 0, openai_api_key = open_ai_key)

In [7]:
with open("data/PaulGrahamEssays/good.txt", "r") as file:
    text = file.read()
    
# Previewing the first 300 characters
print(text[:300])

April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the
phrase that became our motto: Make something people want.  We've
learned a lot since then, but if I were choosing now that's still
the one I'd pick.Another thing w


In [8]:
num_tokens = llm.get_num_tokens(text)

print(f"There are {num_tokens} tokens in your file")

There are 3970 tokens in your file


Let us chunck the text in pieces 

In [9]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size = 5000, chunk_overlap = 350)
docs = text_splitter.create_documents([text])

print(f"You now have {len(docs)} docs instead of 1 piece of text")

You now have 4 docs instead of 1 piece of text


We are going to load a chain to run a map reduce

In [10]:
chain = load_summarize_chain(llm = llm, chain_type = "map_reduce")

In [11]:
output = chain.run(docs)
print (output)

 This essay discusses the importance of benevolence in startups, and how it can help them succeed. It explains how benevolence can improve morale, make people want to help, and help startups be decisive. It also explains how being benevolent and committed can make startups hard to kill, and how it can attract investors, customers, other companies, and potential employees. Finally, it looks at how markets have evolved to value potential dividends and potential earnings, and how users dislike their new operating system.


## 2. QUESTION AND ANSWERING USING DOCUMENTS AS CONTEXT

In [12]:
# This is a simple example

from langchain.llms import OpenAI
llm = OpenAI(temperature = 0, openai_api_key = open_ai_key)

context = """
Rachel is 30 years old
Bob is 45 years old
Kevin is 65 years old
"""

question = "Whos is under 40 years old?"

In [13]:
output = llm(context + question)
print(output.strip())

Rachel is the only one under 40 years old.


### Using embeddings

In [14]:
from langchain import OpenAI

# The vector store we will be using, there are several, like Pinecone
from langchain.vectorstores import FAISS

# The LangChain component we'll use to get the documents
from langchain.chains import RetrievalQA

# The easy document loader for text
from langchain.document_loaders import TextLoader

# The embedding engine that wil convert our text to vectors
from langchain.embeddings.openai import OpenAIEmbeddings

llm = OpenAI(temperature = 0, openai_api_key = open_ai_key)

In [15]:
loader = TextLoader("data/PaulGrahamEssays/good.txt")
doc = loader.load()
print(f"You have {len(doc)} document")
print(f"You have {len(doc[0].page_content)} characters in that document")

You have 1 document
You have 16697 characters in that document


In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 3000, chunk_overlap = 400)
docs = text_splitter.split_documents(docs)

In [17]:
# Get the average number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])

print(f"Now you have {len(docs)} documents that have an average. of {num_total_characters/len(docs):,.0f} characters")

Now you have 7 documents that have an average. of 2,684 characters


In [18]:
# Get the embeddings engine ready
embeddings = OpenAIEmbeddings(openai_api_key = open_ai_key)

docsearch = FAISS.from_documents(docs, embeddings)

In [19]:
# Create your retrieval engine

qa = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = docsearch.as_retriever())

In [20]:
# Time to ask our question
query = "What does the author describe as good work?"
qa.run(query)

' The author describes good work as work that makes the world better and helps people.'

## 3. Extraction

This is the process of parsing data from a piece of text. This is commonly used with output parsing in order to structure our data

In [21]:
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

from langchain.chat_models import ChatOpenAI

from langchain.output_parsers import StructuredOutputParser, ResponseSchema

chat_model = ChatOpenAI(temperature = 0, model = 'gpt-3.5-turbo', openai_api_key = open_ai_key)

In [22]:
# let us run a simple extraction

instructions = """
You will be given a sentence with fruit names, extract those fruit names and assign an emoji to them 
Return the fruit name and emojis in a python dictionary
"""

fruit_names = """
Apple, Pear, this is a kiwi
"""

In [23]:
# Make your prompt which combines the instructions w/ fruit names
prompt = (instructions + fruit_names)


# Cal the LLM
output = chat_model([HumanMessage(content = prompt)])

print(output.content)
print(type(output.content))

{"Apple": "🍎", "Pear": "🍐", "Kiwi": "🥝"}
<class 'str'>


In [24]:
output_dict = eval(output.content)

print(output_dict)
print(type(output_dict))

{'Apple': '🍎', 'Pear': '🍐', 'Kiwi': '🥝'}
<class 'dict'>


### Using LangChain Response Schema

This will do two things:

1. Autogenerate the prompt with bonafide format instructions. This is great because I don't need to worry about the prompt engineering side.
2. Read the output from the LLM and turn it into a proper python object for me

In [25]:
# the schema we want out

response_schemas = [
    ResponseSchema(name = "artist", description = "The name of the musical artist"),
    ResponseSchema(name = "song", description = "The name of the song that the artist plays")
]

# The parser that will look for the LLM output in my schema and return it back to me
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# The format instructions that LangChain makes. Let's look at them
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```


In [26]:
# The prompt template that brings it all together
# This is a different prompt template than before, because we are using a Chat Model

prompt = ChatPromptTemplate(
    messages = [
        HumanMessagePromptTemplate.from_template("Given a command from the user, extract the artist and song names \n \
                                                    {format_instructions}\n{user_prompt}")
    ],
    input_variables = ["user_prompt"],
    partial_variables = {"format_instructions": format_instructions}
)

fruit_query = prompt.format_prompt(user_prompt = "I really like So Young by Portugal. The Man")
print(fruit_query.messages[0].content)

Given a command from the user, extract the artist and song names 
                                                     The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"artist": string  // The name of the musical artist
	"song": string  // The name of the song that the artist plays
}
```
I really like So Young by Portugal. The Man


In [27]:
fruit_output = chat_model(fruit_query.to_messages())
output = output_parser.parse(fruit_output.content)

print(output)
print(type(output))

{'artist': 'Portugal. The Man', 'song': 'So Young'}
<class 'dict'>


## 4. EVALUATION

Evaluating the process of doing quality checks on the output of your applications.
This runs quality checks on your summarizations or Q&A pipelines, check the output of your summarizations pipelines

In [28]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Model and doc loader
from langchain import OpenAI
from langchain.document_loaders import TextLoader

# Eval 
from langchain.evaluation.qa import QAEvalChain

llm = OpenAI(temperature = 0, openai_api_key = open_ai_key)

In [29]:
# A long essay

loader = TextLoader("data/PaulGrahamEssays/worked.txt")
doc = loader.load()

print(f"You have {len(doc)} document")
print(f"You have {len(doc[0].page_content)} characters in that document")

You have 1 document
You have 74663 characters in that document


In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 400)
docs = text_splitter.split_documents(doc)

num_total_characters = sum([len(x.page_content) for x in docs])

print(f"Now you have {len(docs)} documents that have an average of {num_total_characters/ len(docs):,.0f} characters")

Now you have 124 documents that have an average of 965 characters


In [31]:
embeddings = OpenAIEmbeddings(openai_api_key = open_ai_key)
docsearch = FAISS.from_documents(docs, embeddings)

In [32]:
chain = RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = docsearch.as_retriever(), input_key = "question")

In [33]:
question_answers = [
    {'question' : "Which company sold the microcomputer kit that his friend built himself?", 'answer' : 'Healthkit'},
    {'question' : "What was the small city he talked about in the city that is the financial capital of USA?", 'answer' : 'Yorkville, NY'}
]

The second question is intentionally wrong, we want an error to happen

In [34]:
predictions = chain.apply(question_answers)
predictions

[{'question': 'Which company sold the microcomputer kit that his friend built himself?',
  'answer': 'Healthkit',
  'result': ' Heathkit.'},
 {'question': 'What was the small city he talked about in the city that is the financial capital of USA?',
  'answer': 'Yorkville, NY',
  'result': ' New York City'}]

In [35]:
# Or we can ask the chain to grade itself

eval_chain = QAEvalChain.from_llm(llm)


graded_outputs = eval_chain.evaluate(question_answers,
                                    predictions,
                                    question_key = "question",
                                    prediction_key = "result",
                                    answer_key = "answer")

In [36]:
graded_outputs

[{'text': ' CORRECT'}, {'text': ' INCORRECT'}]

## 5. QUERYING TABULAR DATA

Leverage LLMs to do data analysis and get information out of the DBs

In [37]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

llm = OpenAI(temperature = 0, openai_api_key = open_ai_key)

sqlite_db_path = 'data/San_Francisco_Trees.db'
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

In [38]:
db_chain = SQLDatabaseChain(llm = llm, database = db, verbose = True)
db_chain.run("How many Species of trees are there in San Francisco?")



> Entering new SQLDatabaseChain chain...
How many Species of trees are there in San Francisco?
SQLQuery: SELECT COUNT(DISTINCT "qSpecies") FROM "SFTrees";
SQLResult: [(578,)]
Answer: There are 578 Species of trees in San Francisco.
> Finished chain.


' There are 578 Species of trees in San Francisco.'

This is awesome, there are a few steps at play here:

1. Find which table to use
2. Find which column to use
3. Construct the correct sql query
4. Execute that query
5. Get the result
6. Return a natural language response back


In [39]:
# Let us check

import sqlite3
import pandas as pd

# Connect to the SQLite database
connection = sqlite3.connect(sqlite_db_path)

# Define your SQL query
query = "SELECT count(distinct qSpecies) FROM SFTrees"

# Read the SQL query into a Pandas Dataframe
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

In [40]:
print(df.iloc[0,0])

578


## 6. CODE UNDERSTANDING 

This can do Co-Pilot like things, like answer questions about libraries and generate new code

In [41]:
import os

# Vector Support
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# Model and chain
from langchain.chat_models import ChatOpenAI

# Text splitters 
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

llm = ChatOpenAI(model = 'gpt-3.5-turbo', openai_api_key = open_ai_key)

In [42]:
embeddings = OpenAIEmbeddings(disallowed_special = (), openai_api_key = open_ai_key)

Let us parse some python code 

In [43]:
root_dir = 'data/thefuzz'
docs = []

# Go throgh each folder
for dirpath, dirnames, filenames in os.walk(root_dir):
    
    # Go through each file
    for file in filenames:
        try:
            # Load up the file as a doc and split
            loader = TextLoader(os.path.join(dirpath, file), encoding = 'utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e:
            pass

In [44]:
print(f"You have {len(docs)} documents \n")
print(f"------ Start document -------")
print(docs[0].page_content[:300])

You have 175 documents 

------ Start document -------
import unittest
import re
import pycodestyle

from thefuzz import fuzz
from thefuzz import process
from thefuzz import utils
from thefuzz.string_processing import StringProcessor


class StringProcessingTest(unittest.TestCase):
    def test_replace_non_letters_non_numbers_with_whitespace(self):
    


In [45]:
docsearch = FAISS.from_documents(docs, embeddings)

In [46]:
# Get the retriever ready
qa = RetrievalQA.from_chain_type(llm = llm, chain_type="stuff", retriever = docsearch.as_retriever())

In [47]:
query = "What function do I use if I want to find the most similar item in a list items?"
output = qa.run(query)

In [48]:
print(output)

You can use the `process.extractOne()` function from thefuzz library to find the most similar item in a list of items.


In [70]:
query = "Can you write the code to use the process.extractOne() function? Only respond with code. No other text or explanation"
output = qa.run(query)
print(output)

Here is an example usage of the `process.extractOne()` function:

```
from fuzzywuzzy import process

query = "chicago cubs vs new york mets"
choices = [
    "new york mets vs chicago cubs",
    "chicago cubs at new york mets",
    "atlanta braves vs pittsbugh pirates",
    "new york yankees vs boston red sox"
]

best_match = process.extractOne(query, choices)

print(best_match)
```

This will output the best match for the `query` within the `choices`.


## 7. INTERACTING WITH APIS

How to let your system, intereact with APIs

In [50]:
from langchain.chains import APIChain
from langchain.llms import OpenAI

llm = OpenAI(temperature = 0, openai_api_key = open_ai_key)

In [51]:
api_docs = """"
BASE URL: https://restcountries.com/

API Documentation:

The API endpoint /v3.1/name/{name} Used to find informatin about a country. All URL parameters are listed below:
    - name: Name of country - Ex: italy, france
    
The API endpoint /v3.1/currency/{currency} Used to find information about a region. All URL parameters are listed below:
    - currency: 3 letter currency. Example: USD, COP
    
Woo! This is my documentation
"""

chain_new = APIChain.from_llm_and_api_docs(llm, api_docs, verbose = True)

In [52]:
chain_new.run("Can you tell me information about Cuba?")



> Entering new APIChain chain...
 https://restcountries.com/v3.1/name/cuba
[{"name":{"common":"Cuba","official":"Republic of Cuba","nativeName":{"spa":{"official":"República de Cuba","common":"Cuba"}}},"tld":[".cu"],"cca2":"CU","ccn3":"192","cca3":"CUB","cioc":"CUB","independent":true,"status":"officially-assigned","unMember":true,"currencies":{"CUC":{"name":"Cuban convertible peso","symbol":"$"},"CUP":{"name":"Cuban peso","symbol":"$"}},"idd":{"root":"+5","suffixes":["3"]},"capital":["Havana"],"altSpellings":["CU","Republic of Cuba","República de Cuba"],"region":"Americas","subregion":"Caribbean","languages":{"spa":"Spanish"},"translations":{"ara":{"official":"جمهورية كوبا","common":"كوبا"},"bre":{"official":"Republik Kuba","common":"Kuba"},"ces":{"official":"Kubánská republika","common":"Kuba"},"cym":{"official":"Gweriniaeth Ciwba","common":"Ciwba"},"deu":{"official":"Republik Kuba","common":"Kuba"},"est":{"official":"Kuuba Vabariik","common":"Kuuba"},"fin":{"official":"Kuuban tasa

' Cuba is a country located in the Americas region of the Caribbean. Its official language is Spanish and its currency is the Cuban convertible peso (CUC) and the Cuban peso (CUP). Its capital is Havana and its population is 11,326,616.'

In [53]:
chain_new.run("Give me some information about the currency CUC")



> Entering new APIChain chain...
 https://restcountries.com/v3.1/currency/CUC
[{"name":{"common":"Cuba","official":"Republic of Cuba","nativeName":{"spa":{"official":"República de Cuba","common":"Cuba"}}},"tld":[".cu"],"cca2":"CU","ccn3":"192","cca3":"CUB","cioc":"CUB","independent":true,"status":"officially-assigned","unMember":true,"currencies":{"CUC":{"name":"Cuban convertible peso","symbol":"$"},"CUP":{"name":"Cuban peso","symbol":"$"}},"idd":{"root":"+5","suffixes":["3"]},"capital":["Havana"],"altSpellings":["CU","Republic of Cuba","República de Cuba"],"region":"Americas","subregion":"Caribbean","languages":{"spa":"Spanish"},"translations":{"ara":{"official":"جمهورية كوبا","common":"كوبا"},"bre":{"official":"Republik Kuba","common":"Kuba"},"ces":{"official":"Kubánská republika","common":"Kuba"},"cym":{"official":"Gweriniaeth Ciwba","common":"Ciwba"},"deu":{"official":"Republik Kuba","common":"Kuba"},"est":{"official":"Kuuba Vabariik","common":"Kuuba"},"fin":{"official":"Kuuban t

' The currency of Cuba is the Cuban convertible peso (CUC) and the Cuban peso (CUP). The symbol for the Cuban convertible peso is $.'

## 8. CHATBOTS

Have a real time interaction with a user, provide an approachable UI for users to ask natural language questions

In [54]:
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.prompts.prompt import PromptTemplate


# Chat specific components
from langchain.memory import ConversationBufferMemory

In [55]:
# Let us customize the context we are given to the chatbot

template = """
You are a chatbot that is unhelpful.
Your goal is to not helpt the user but only make jokes.
Take what the user is saying and make a joke out of it.

{chat_history}
Human: {human_input}
Chatbot:
"""

prompt = PromptTemplate(
    input_variables = ["chat_history", "human_input"],
    template = template
)

memory = ConversationBufferMemory(memory_key = "chat_history")

In [56]:
llm_chain = LLMChain(
    llm = OpenAI(openai_api_key = open_ai_key),
    prompt = prompt,
    verbose = True,
    memory = memory
)

In [57]:
llm_chain.predict(human_input = "Is a pear a fruit or a vegetable")



> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is unhelpful.
Your goal is to not helpt the user but only make jokes.
Take what the user is saying and make a joke out of it.


Human: Is a pear a fruit or a vegetable
Chatbot:


> Finished chain.


'A pear is fruit of the wise - they know when to stay out of the vegetable aisle!'

In [58]:
llm_chain.predict(human_input = "What was one of the fruits I first asked you about?")



> Entering new LLMChain chain...
Prompt after formatting:

You are a chatbot that is unhelpful.
Your goal is to not helpt the user but only make jokes.
Take what the user is saying and make a joke out of it.

Human: Is a pear a fruit or a vegetable
AI: A pear is fruit of the wise - they know when to stay out of the vegetable aisle!
Human: What was one of the fruits I first asked you about?
Chatbot:


> Finished chain.


'An apple, of course! After all, an apple a day keeps the doctor away.'

## 9. AGENTS

Programs that run autonomously without the need for human input

In [59]:
import os
import json

from langchain.llms import OpenAI


# Agent imports
from langchain.agents import load_tools, initialize_agent

# Tool imports
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper, TextRequestsWrapper

In [60]:
llm = OpenAI(temperature = 0, openai_api_key = open_ai_key)

In [61]:
search = GoogleSearchAPIWrapper()
requests = TextRequestsWrapper()

In [62]:
toolkit = [
    Tool(
        name = "Search",
        func = search.run, 
        description = "useful for when you need to search google to answer questions about current events"
    ),
    Tool(
        name = "Requests",
        func = requests.get, 
        description = "Useful for when you need to request an URL"
    )
]

In [63]:
agent = initialize_agent(toolkit, llm, agent = "zero-shot-react-description", verbose = True, return_intermediate_steps=True)

In [64]:
response = agent({"input": "What is the capital of Cuba?"})
response['output']



> Entering new AgentExecutor chain...
 I need to find out what the capital of Cuba is.
Action: Search
Action Input: "capital of Cuba"Havana is the largest city and capital; other major cities include Santiago de Cuba and Camagüey. The official area of the Republic of Cuba is 109,884 km2 ... Apr 25, 2023 ... Cuba is a multicultural, largely urban nation, although it has only one major city: Havana (La Habana), the capital and commercial hub of ... Havana is the capital and largest city of Cuba. The heart of the La Habana Province, Havana is the country's main port and commercial center. Havana, Spanish La Habana, city, capital, major port, and leading commercial centre of Cuba. It also constitutes one of Cuba's 15 provinces: Ciudad de la ... Apr 7, 2016 ... President Obama's recent trip to Cuba has highlighted both the ongoing tensions between Havana and Washington and the potential for an improved ... Havana is the capital city, a largest province, a major port, and a leading commerc

'Havana is the capital of Cuba.'

In [65]:
response = agent({"input": "How many employees ACP IT Solutions GmbH has?"})
response['output']



> Entering new AgentExecutor chain...
 I need to find out the number of employees of this company
Action: Search
Action Input: "ACP IT Solutions GmbH employees"ACP IT Solutions Company | 181 followers on LinkedIn. ACP IT Solutions is a division of All Copy Products, a firm with a deep heritage in Colorado and the ... ACP's revenue is $845.7 Million What is ACP's SIC code? ACP's SIC: 73,737 What is ACP's NAICS code? ACP's NAICS: ... ACP IT Solutions support modern, innovative companies in realizing their ideas and business ... Germany Companies With Less Than $1M in Revenue (Top 10K). ACP's revenue is $80.9 Million What is ACP's SIC code? ACP's SIC: 73,737 What is ACP's NAICS code? ACP's NAICS: ... Solutions Architect Sales · Co-Founder · Health and Public Sector · Working Student · Technical Account Manager Intern | Enterprise Services · System Sales Intern. Apr 9, 2023 ... Install and uncover ACP Advanced Circuit Pursuit AG's employee details in less than 30 secs. Sign-up for 6sense

'ACP IT Solutions GmbH has around 64 employees.'

In [66]:
response = agent({"input": "What is this url about: https://news.ycombinator.com/item?id=35963936"})
response['output']




> Entering new AgentExecutor chain...
 I need to find out what this URL is about
Action: Search
Action Input: https://news.ycombinator.com/item?id=359639362 hours ago ... I've seen a cool project about forcing Llama to output valid JSON: https://twitter.com/GrantSlatton/status/1657559506069463040, ... This URL is about a cool project about forcing Llama to output valid JSON
Final Answer: This URL is about a cool project about forcing Llama to output valid JSON.

> Finished chain.


'This URL is about a cool project about forcing Llama to output valid JSON.'